# Packages

In [ ]:
install.packages("pracma")
install.packages("numDeriv")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



# Modeling Code

In [ ]:
library(pracma)


f <- function(x, l, m, s) {
    FR <- ((l^x)*((1-l)^(1-x))+l-1)/((2*l)-1) # CDF Continuous Bernoulli
    e1 <- pi*(FR -.5)
    e2 <- -(1-l)^(1-x)
    e3 <- log((1-l)/l)
    numerator <- exp(-((m-tan(e1))^2)/(2*(s^2))) * sqrt(pi/2) * e2 * (l^x) * e3 * ((sec(e1))^2)
    denominator <- ((2 * l)-1) * s
    NCBC <- numerator/denominator
}

x1 <- seq(0,1,by=0.001)

plot (x1,f(x1,0.3,-2.0,0.5),
      ylim=c(0,18.5),
      xlim=c(0,1),
      ylab="f(x)",
      xlab="",
      type="l",
      lty=1,
      col="black",
      main = "PDF of NCBC")
lines(x1,f(x1,0.6,-1.5,0.5),
      lty=2,
      lwd=2,
      col="#E1981A")
lines(x1,f(x1,0.15,0.2,0.45),
      lty=3,
      lwd=2,
      col="#D07E14")
lines(x1,f(x1,0.6,-0.3,2.0),
      lty=4,
      lwd=2,
      col="#BD6510")
lines(x1,f(x1,0.9,0.7,0.5),
      lty=5,
      lwd=2,
      col="#A94D0D")
lines(x1,f(x1,0.9,1.5,0.5),
      lty=6,
      lwd=2,
      col="#93360B")
lines(x1,f(x1,0.55,0.0,0.2),
      lty=7,
      lwd=2,
      col="#7D1E08")

col <- c("black","#E1981A","#D07E14","#BD6510","#A94D0D", "#93360B","#7D1E08")

#Legend
legend(0.15,19,lwd=2,
       c(expression(lambda==0.3~~~~mu==-2.0~~~sigma==0.5),
         expression(lambda==0.6~~~~mu==-1.5~~~sigma==0.5),
         expression(lambda==0.15~~~mu==0.2~~~~sigma==0.45),
         expression(lambda==0.6~~~~mu==-0.3~~~sigma==2.0),
         expression(lambda==0.9~~~~mu==0.7~~~~sigma==0.5),
         expression(lambda==0.9~~~~mu==1.5~~~~sigma==0.5),
         expression(lambda==0.55~~~mu==0.0~~~~sigma==0.2)),
       lty=1:6,cex=.65, col=col, bty="n" )

# Apply label to bottom of plot
mtext(text="x", side=1, line=1.75)

# Reset the plot settings
par(mfrow = c(1, 1))


# Original Code

In [ ]:
library(stats)
library(pracma)
library(stats4)

set.seed(4)
df2 <- read.csv("abalone.csv")
df2_main_col <- df2[, c(2)]
df2_col <- df2_main_col

#-----------------------------------------------------------------------------------------------
##NCBC section
ncbc_pdf <- function(params, x) {
  lambda <- params[1]
  mu <- params[2]
  sigma <- params[3]
  f_r = (2 * atanh(1 - 2 * lambda) * (lambda ^ x) *((1 - lambda) ^ (1 - x))) / (1-(2 * lambda))
  F_r = ((lambda^x) * ((1 - lambda) ^ (1 - x)) + lambda - 1) / ((2 * lambda) - 1)
  z <- exp((-(tan(pi * (F_r - 0.5))-mu)^2) / (2 * sigma^2))
  numerator <- z * pi * sec(pi * (F_r - 0.5)) * f_r
  denominator <- sqrt(2 * pi * (sigma^2))
  f = numerator/denominator
}

#ncbc cdf, used for ks test
ncbc_cdf <- function(params, x) {
  lambda <- params[1]
  mu <- params[2]
  sigma <- params[3]
  f_r <- (2 * atanh(1 - 2 * lambda) * (lambda ^ x) *((1 - lambda) ^ (1 - x))) / (1-(2 * lambda))
  F_r <- ((lambda^x) * ((1 - lambda) ^ (1 - x)) + lambda - 1) / ((2 * lambda) - 1)
  f <- (1 / 2) * (1 + erf((tan(pi * (F_r - (1 / 2))) - mu) / (sigma * sqrt(2))))
}

ncbc_log_likelihood_v2 <- function(params, x) {
  lambda <- params[1]
  mu <- params[2]
  sigma <- params[3]
  n <- length(x)
  F_r = ((lambda^x) * ((1 - lambda) ^ (1 - x)) + lambda - 1) / ((2 * lambda) - 1)
  f_r = (2 * atanh(1 - 2 * lambda) * (lambda ^ x) *((1 - lambda) ^ (1 - x))) / (1-(2 * lambda))

  log_expr <- -(1/2) * log(2*pi*(sigma^2)) - (((tan(pi * (F_r - 0.5))-mu)^2) / (2 * (sigma^2))) + log(pi * sec(pi * (F_r - 0.5))) + log(f_r)

  log_lik_expr <- -(n/2) * log(2*pi*(sigma^2)) - (sum((tan(pi * (F_r - 0.5))-mu)^2) / (2 * (sigma^2))) + sum(log(pi * sec(pi * (F_r - 0.5)))) + sum(log(f_r))
  n_log_lik <- -log_lik_expr
}

bfgs_optimization_ncbc <- function(data) {
  init_params <- c(0.9, 0.3, 0.2)

  result <- optim(
    par = init_params,
    fn = ncbc_log_likelihood_v2,
    x = data,
    method = "L-BFGS-B",
    lower = c(0.1, -Inf, 0.1), #lower bounds
    upper = c(0.9, Inf, Inf), #upper bounds
    control = list(maxit = 1000, trace = 3),
    hessian = TRUE
  )

  return(result)
}
#---------------------------------------------------------------------------------------------
##CB Section

cb_pdf <- function(params, x) {
    lambda <- params[1]
    cLamb = (2 * atanh(1 - 2 * lambda)) / (1 - 2*lambda)
    f_r = (2 * atanh(1 - 2 * lambda) * (lambda ^ x) *((1 - lambda) ^ (1 - x))) / (1-(2 * lambda))
}

#cb cdf, used for ks test
cb_cdf <- function(params, x) {
  lambda <- params[1]
  F_r = ((lambda^x) * ((1 - lambda) ^ (1 - x)) + lambda - 1) / ((2 * lambda) - 1)
}

cb_log_likelihood <- function(params, x) {
    lambda <- params[1]
    x = data
    cLamb = (2 * atanh(1 - 2 * lambda)) / (1 - 2*lambda)
    f = (2 * atanh(1 - 2 * lambda) * (lambda ^ x) *((1 - lambda) ^ (1 - x))) / (1-(2 * lambda))
    l = prod(f)
    ll = -log(l)
}

bfgs_optimization_cb <- function(params, x) {
  init_params <- c(0.2)

  result <- optim(
    par = init_params,
    fn = cb_log_likelihood,
    x = data,
    method = "L-BFGS-B",
    lower = c(0.1), #lower bounds
    upper = c(0.9), #upper bounds
    control = list(maxit = 1000, trace = 3),
    hessian = TRUE
  )

  return(result)
}
#----------------------------------------------------------------------------------------------
##Normizzl disribbel
normal_pdf <- function(params, x) {
  mu <- params[1]
  sigma <- params[2]
  f <- dnorm(x, mean = mu, sd = sigma)
}

normal_cdf <- function(params, x) {
  mu <- params[1]
  sigma <- params[2]
  f <- pnorm(x, mean = mu, sd = sigma)
}

normal_log_likelihood <- function(params, x) {
  mu <- params[1]
  sigma <- params[2]
  f <- dnorm(x, mean = mu, sd = sigma)
  l <- prod(f)
  ll <- -log(l)
}

bfgs_optimization_norm <- function(params, x) {
  init_params <- c(2, 1)

  result <- optim(
    par = init_params,
    fn = normal_log_likelihood,
    x = data,
    method = "BFGS",
    control = list(maxit = 1000, trace = 3),
    hessian = TRUE
  )

  return(result)
}
#---------------------------------------------------------------------------------------------
#cauchussy distribussy

cauch_pdf <- function(params, x) {
  mu <- params[1]
  sigma <- params[2]
  f <- dcauchy(x, mu, sigma)
}

cauch_cdf <- function(params, x) {
  mu <- params[1]
  sigma <- params[2]
  f <- pcauchy(x, mu, sigma)
}

cauch_log_likelihood <- function(params, x) {
  mu <- params[1]
  sigma <- params[2]
  f <- dcauchy(x, mu, sigma)
  l <- prod(f)
  ll <- -log(l)
}

bfgs_optimization_cauch <- function(params, x) {
  init_params <- c(0.6, 0.1)

  result <- optim(
    par = init_params,
    fn = cauch_log_likelihood,
    x = data,
    method = "L-BFGS-B",
    lower = c(-Inf, 0),
    upper = c(Inf, Inf),
    control = list(maxit = 1000, trace = 3),
    hessian = TRUE
  )

  return(result)
}

#----------------------------------------------------------------------------------------------
##data section
##cleaning data

#using as placeholder for negative cov values
epsilon <- 1e-5
data <- sample(df2_col, 100)

##run BFGS optimization
optim_res_ncbc <- bfgs_optimization_ncbc(data)
optim_res_norm <- bfgs_optimization_norm(data)
optim_res_cauch <- bfgs_optimization_cauch(data)

optim_res_cb <- bfgs_optimization_cb(data)

print(optim_res_ncbc)
print(optim_res_norm)
print(optim_res_cauch)

print(optim_res_cb)

#get params of optimized params (for modeling)
res_params_ncbc <- optim_res_ncbc$par
res_params_norm <- optim_res_norm$par
res_params_cauch <- optim_res_cauch$par

res_params_cb <- optim_res_cb$par
#covariance matrices section
hess_ncbc <- optim_res_ncbc$hessian
cov_ncbc <- solve(hess_ncbc + diag(epsilon, nrow(hess_ncbc)))
cat("COVARIANCE MATRIX OF NCBC\n")
print(cov_ncbc)
cat("STANDARD ERRORS (LAMBDA, MU, SIGMA)\n")
print(diag(sqrt(cov_ncbc)))

hess_norm <- optim_res_norm$hessian
cov_norm <- solve(hess_norm + diag(epsilon, nrow(hess_norm)))
cat("COVARIANCE MATRIX OF NORM\n")
print(cov_norm)
cat("STANDARD ERRORS (MU, SIGMA)\n")
print(diag(sqrt(cov_norm)))

hess_cauch <- optim_res_cauch$hessian
cov_cauch <- solve(hess_cauch + diag(epsilon, nrow(hess_cauch)))
cat("COVARIANCE MATRIX OF CAUCH\n")
print(cov_cauch)
cat("STANDARD ERRORS (X_0, GAMMA)\n")
print(diag(sqrt(cov_cauch)))
#----------------------------------------------------------------------------------------------
#tests section
#define empirical cdf for use in ks statistic
ecdf_data <- ecdf(data)
x_values <- seq(min(data), max(data), length.out = 1000)
ecdf_values <- ecdf_data(x_values)
n <- length(data)

#for computing p-value of ks test, emulates pkstwo()
pkstwo <- function(x) {
  if (x < 0) return(0)
  return(1 - (1 - x)^(2))
}

ncbc_likelihood_val <- exp(optim_res_ncbc$value)
#---------------------------------------------------------------------------------------------
#ncbc tests
cat("NCBC AIC\n")
ncbc_aic <- 2 * 3 + 2 * optim_res_ncbc$value
print(ncbc_aic)

cat("NCBC BIC\n")
ncbc_bic <- log(length(data)) * 3 + 2 * optim_res_ncbc$value
print(ncbc_bic)

ncbc_theoretical_values <- ncbc_cdf(res_params_ncbc, x_values)
ncbc_differences <- abs(ecdf_values - ncbc_theoretical_values)
ncbc_ks <- max(ncbc_differences)
ncbc_pval <- pkstwo(ncbc_ks)

cat("K-S STATISTIC", ncbc_ks, "\n", "P-VAL", ncbc_pval, "\n")
#----------------------------------------------------------------------------------------------
norm_likelihood_val <- exp(optim_res_norm$value)
cat("NORM AIC\n")
norm_aic <- 2 * 1 + 2 * optim_res_norm$value
print(norm_aic)
cat("NORM BIC\n")
norm_bic <- log(length(data)) * 1 + 2 * optim_res_norm$value
print(norm_bic)

norm_theoretical_values <- normal_cdf(res_params_norm, x_values)
norm_differences <- abs(ecdf_values - norm_theoretical_values)
norm_ks <- max(norm_differences)
norm_pval <- pkstwo(norm_ks)

cat("K-S STATISTIC", norm_ks, "\n", "P-VAL", norm_pval, "\n")
#--------------------------------------------------------------------------------------------
cauch_likelihood_val <- exp(optim_res_cauch$value)
cat("CAUCH AIC\n")
cauch_aic <- 2 * 1 + 2 * optim_res_cauch$value
print(cauch_aic)
cat("CAUCH BIC\n")
cauch_bic <- log(length(data)) * 1 + 2 * optim_res_cauch$value
print(cauch_bic)

cauch_theoretical_values <- cauch_cdf(res_params_cauch, x_values)
cauch_differences <- abs(ecdf_values - cauch_theoretical_values)
cauch_ks <- max(cauch_differences)
cauch_pval <- pkstwo(cauch_ks)

cat("K-S STATISTIC", cauch_ks, "\n", "P-VAL", cauch_pval, "\n")
#--------------------------------------------------------------------------------------------
print(min(data))

#modeling section

pdf("abalone_lengths.pdf", width = 10)
x1 <- seq(0, 1, by=0.01)
hist(data, freq = FALSE, breaks = 20, main = "Shell Lengths in Abalone Dataset",
    xlab = "Shell Lengths",xlim = c(0.1,0.9), ylab = "Density", ylim = c(0,5))
lines(x1, ncbc_pdf(res_params_ncbc,x1),col = "red", type = "l", lty = 2, lwd = 2)
lines(x1, cauch_pdf(res_params_cauch,x1), col = "orange", type = "l", lty = 5, lwd = 2)
lines(x1, normal_pdf(res_params_norm,x1), col = "purple", type = "l", lty = 6, lwd = 2)
legend(0.15,4.5, legend = c("NCBC","CAUCHY","NORM"),
    col = c("red","orange","purple"), lty = c(2,5,6), lwd = 2)
dev.off()


N = 3, M = 5 machine precision = 2.22045e-16
At X0, 1 variables are exactly at the bounds
At iterate     0  f=       3189.1  |proj g|=        33438
At iterate    10  f =      -62.644  |proj g|=       0.75404

iterations 18
function evaluations 22
segments explored during Cauchy searches 19
BFGS updates skipped 0
active bounds at final generalized Cauchy point 0
norm of the final projected gradient 0.00251725
final function value -62.8396

F = -62.8396
final  value -62.839553 
converged
initial  value 203.553953 


Warning message in dnorm(x, mean = mu, sd = sigma):
“NaNs produced”
Warning message in dnorm(x, mean = mu, sd = sigma):
“NaNs produced”
Warning message in dnorm(x, mean = mu, sd = sigma):
“NaNs produced”
Warning message in dnorm(x, mean = mu, sd = sigma):
“NaNs produced”
Warning message in dnorm(x, mean = mu, sd = sigma):
“NaNs produced”
Warning message in dnorm(x, mean = mu, sd = sigma):
“NaNs produced”
Warning message in dnorm(x, mean = mu, sd = sigma):
“NaNs produced”
Warning message in dnorm(x, mean = mu, sd = sigma):
“NaNs produced”
Warning message in dnorm(x, mean = mu, sd = sigma):
“NaNs produced”
Warning message in dnorm(x, mean = mu, sd = sigma):
“NaNs produced”
Warning message in dnorm(x, mean = mu, sd = sigma):
“NaNs produced”
Warning message in dnorm(x, mean = mu, sd = sigma):
“NaNs produced”
Warning message in dnorm(x, mean = mu, sd = sigma):
“NaNs produced”
Warning message in dnorm(x, mean = mu, sd = sigma):
“NaNs produced”


iter  10 value -69.700572


Warning message in dnorm(x, mean = mu, sd = sigma):
“NaNs produced”
Warning message in dnorm(x, mean = mu, sd = sigma):
“NaNs produced”
Warning message in dnorm(x, mean = mu, sd = sigma):
“NaNs produced”
Warning message in dnorm(x, mean = mu, sd = sigma):
“NaNs produced”


final  value -70.502477 
converged
N = 2, M = 5 machine precision = 2.22045e-16
At X0, 0 variables are exactly at the bounds
At iterate     0  f=      -34.283  |proj g|=       372.39

iterations 6
function evaluations 9
segments explored during Cauchy searches 7
BFGS updates skipped 0
active bounds at final generalized Cauchy point 0
norm of the final projected gradient 2.97122e-06
final function value -49.9881

F = -49.9881
final  value -49.988085 
converged
N = 1, M = 5 machine precision = 2.22045e-16
At X0, 0 variables are exactly at the bounds
At iterate     0  f=       9.3248  |proj g|=          0.7

iterations 5
function evaluations 25
segments explored during Cauchy searches 5
BFGS updates skipped 0
active bounds at final generalized Cauchy point 0
norm of the final projected gradient 1.62297e-06
final function value -0.0649044

F = -0.0649044
   in the last line search
final  value -0.064904 
converged
$par
[1] 0.4175299 0.1683765 0.4396343

$value
[1] -62.83955

$counts
functi

Warning message in sqrt(cov_ncbc):
“NaNs produced”


[1] 0.05526962 0.10919370 0.03117814
COVARIANCE MATRIX OF NORM
              [,1]          [,2]
[1,]  1.429413e-04 -3.392397e-09
[2,] -3.392397e-09  7.142780e-05
STANDARD ERRORS (MU, SIGMA)


Warning message in sqrt(cov_norm):
“NaNs produced”


[1] 0.011955805 0.008451497
COVARIANCE MATRIX OF CAUCH
             [,1]         [,2]
[1,] 1.877784e-04 1.041572e-06
[2,] 1.041572e-06 8.907306e-05
STANDARD ERRORS (X_0, GAMMA)
[1] 0.013703226 0.009437852
NCBC AIC
[1] -119.6791
NCBC BIC
[1] -111.8636
K-S STATISTIC 0.08159328 
 P-VAL 0.1565291 
NORM AIC
[1] -139.005
NORM BIC
[1] -136.3998
K-S STATISTIC 0.08961805 
 P-VAL 0.1712047 
CAUCH AIC
[1] -97.97617
CAUCH BIC
[1] -95.371
K-S STATISTIC 0.1275946 
 P-VAL 0.2389088 
[1] 0.175


agg_record_1433661835 
                    2

# Refactor with numDeriv

In [12]:
library(stats)
library(stats4)
library(numDeriv)
library(pracma)
##### ---- ORIGINAL CODE FOR REFERENCE ---- #####
#set.seed(4)
#df2 <- read.csv("data/abalone.csv")
#df2_main_col <- df2[, c(2)]
#df2_col <- df2_main_col
#data <- sample(df2_col, 100)

# load rds files
data <- readRDS("abalone_shell_lengths_sample_4-4-2.rds")


# x values for plotting and ks testing
x_values <- seq(min(data), max(data), length.out = 1000)
ecdf_data <- ecdf(data)
ecdf_vals <- ecdf_data(x_values)

# pkstwo hard coded because I couldn't get library to work
pkstwo <- function(x) {
  if (x < 0) return(0)
  return(1 - (1 - x)^2)
}

# list of distributions and components
# Distribution definitions
distributions <- list(
  ncbc = list(
    name = "NCBC",
    pdf = function(params, x) {
      lambda <- params[1]; mu <- params[2]; sigma <- params[3]
      F_r <- ((lambda^x) * ((1 - lambda)^(1 - x)) + lambda - 1) / (2 * lambda - 1)
      f_r <- (2 * atanh(1 - 2 * lambda) * (lambda^x) * ((1 - lambda)^(1 - x))) / (1 - 2 * lambda)
      z <- exp(-((tan(pi * (F_r - 0.5)) - mu)^2) / (2 * sigma^2))
      num <- z * pi * sec(pi * (F_r - 0.5)) * f_r
      denom <- sqrt(2 * pi * sigma^2)
      num / denom
    },
    cdf = function(params, x) {
      lambda <- params[1]; mu <- params[2]; sigma <- params[3]
      F_r <- ((lambda^x) * ((1 - lambda)^(1 - x)) + lambda - 1) / (2 * lambda - 1)
      0.5 * (1 + erf((tan(pi * (F_r - 0.5)) - mu) / (sigma * sqrt(2))))
    },
    log_likelihood = function(params, x) {
      lambda <- params[1]; mu <- params[2]; sigma <- params[3]
      F_r <- ((lambda^x) * ((1 - lambda)^(1 - x)) + lambda - 1) / (2 * lambda - 1)
      f_r <- (2 * atanh(1 - 2 * lambda) * (lambda^x) * ((1 - lambda)^(1 - x))) / (1 - 2 * lambda)
      log_lik <- -0.5 * log(2 * pi * sigma^2) - ((tan(pi * (F_r - 0.5)) - mu)^2) / (2 * sigma^2) +
        log(pi * sec(pi * (F_r - 0.5))) + log(f_r)
      -sum(log_lik)
    },
    init = c(0.8, 0.3, 0.2),
    lower = c(0.1, -Inf, 0.1),
    upper = c(0.9, Inf, Inf),
    method = "L-BFGS-B"
  ),

  norm = list(
    name = "Normal",
    pdf = function(params, x) dnorm(x, mean = params[1], sd = params[2]),
    cdf = function(params, x) pnorm(x, mean = params[1], sd = params[2]),
    log_likelihood = function(params, x) -sum(log(dnorm(x, mean = params[1], sd = params[2]))),
    init = c(mean(data), sd(data)),
    method = "BFGS"
  ),

  cauchy = list(
    name = "Cauchy",
    pdf = function(params, x) dcauchy(x, location = params[1], scale = params[2]),
    cdf = function(params, x) pcauchy(x, location = params[1], scale = params[2]),
    log_likelihood = function(params, x) -sum(log(dcauchy(x, location = params[1], scale = params[2]))),
    init = c(median(data), 0.1),
    lower = c(-Inf, 0.001),
    upper = c(Inf, Inf),
    method = "L-BFGS-B"
  )
)

# gradient and hessian helper functions
make_grad_fn <- function(ll_fn, data) {
  function(params) grad(function(p) ll_fn(p, data), params)
}

make_hess_fn <- function(ll_fn, data) {
  function(params) hessian(function(p) ll_fn(p, data), params)
}

# fit distribution using numerical gradient
fit_distribution <- function(dist, data) {
  grad_fn <- make_grad_fn(dist$log_likelihood, data)
  hess_fn <- make_hess_fn(dist$log_likelihood, data)

  optim_args <- list(
    par = dist$init,
    fn = function(p) dist$log_likelihood(p, data),
    gr = grad_fn,
    method = dist$method,
    control = list(maxit = 1000, trace = 3)
  )

  if (dist$method %in% c("L-BFGS-B", "Brent")) {
    optim_args$lower <- dist$lower
    optim_args$upper <- dist$upper
  }

  result <- do.call(optim, optim_args)
  result$hessian <- hess_fn(result$par)
  return(result)
}
# results
evaluate_fit <- function(result, dist_name, data_len, num_params, cdf_func, data, x_vals, ecdf_vals) {
  loglik <- result$value
  aic <- 2 * num_params + 2 * loglik
  bic <- log(data_len) * num_params + 2 * loglik
  epsilon <- max(abs(diag(result$hessian))) * 1e-3
  cov_matrix <- solve(result$hessian + diag(epsilon, nrow(result$hessian)))
  se <- sqrt(diag(cov_matrix))

  theoretical_cdf <- cdf_func(result$par, x_vals)
  ks_stat <- max(abs(ecdf_vals - theoretical_cdf))
  pval <- pkstwo(ks_stat)

  cat(sprintf("=== %s ===\n", dist_name))
  cat("Estimated Parameters:\n"); print(result$par)
  cat("Final Value (NLL):", loglik, "\n")
  cat("Log-Likelihood:", exp(loglik), "\n")
  cat("AIC:", aic, "\n")
  cat("BIC:", bic, "\n")
  cat("K-S Statistic:", ks_stat, "\n")
  cat("P-Value:", pval, "\n")
  cat("Standard Errors:\n"); print(se)
  cat("\n")

  list(aic = aic, bic = bic, ks = ks_stat, pval = pval, se = se, cov = cov_matrix)
}

# testing stuff
results <- lapply(distributions, fit_distribution, data = data)
evaluations <- mapply(
  FUN = evaluate_fit,
  result = results,
  dist_name = names(distributions),
  num_params = sapply(distributions, function(d) length(d$init)),
  cdf_func = lapply(distributions, function(d) d$cdf),
  MoreArgs = list(data_len = length(data), data = data, x_vals = x_values, ecdf_vals = ecdf_vals),
  SIMPLIFY = FALSE
)

# plotting
pdf("shell_lengths_abalone.pdf", width = 10)
hist(data, freq = FALSE, main = "Shell Lengths in Abalone Dataset",
     xlab = "Shell Lengths", ylab = "Density", xlim = c(0, 1), ylim = c(0, 5))
x1 <- seq(0, 1, by = 1e-4)
colors <- c("#000000", "#566E3D", "#FF934F")
lt <- c(1, 4, 6)
i <- 1
for (name in names(distributions)) {
  lines(x1, distributions[[name]]$pdf(results[[name]]$par, x1), col = colors[i], lty = lt[i], lwd = 2)
  i <- i + 1
}
legend(0.15, 4.5, legend = toupper(names(distributions)), col = colors, lty = lt, lwd = 2)
dev.off()

N = 3, M = 5 machine precision = 2.22045e-16
At X0, 0 variables are exactly at the bounds
At iterate     0  f=        50848  |proj g|=   5.6971e+05
At iterate    10  f =      -2463.1  |proj g|=         393.3

iterations 18
function evaluations 24
segments explored during Cauchy searches 19
BFGS updates skipped 0
active bounds at final generalized Cauchy point 0
norm of the final projected gradient 0.01388
final function value -2517.38

F = -2517.38
final  value -2517.384296 
converged
initial  value -2925.666112 


Warning message in dnorm(x, mean = params[1], sd = params[2]):
“NaNs produced”
Warning message in dnorm(x, mean = params[1], sd = params[2]):
“NaNs produced”
Warning message in dnorm(x, mean = params[1], sd = params[2]):
“NaNs produced”


final  value -2925.666171 
converged
N = 2, M = 5 machine precision = 2.22045e-16
At X0, 0 variables are exactly at the bounds
At iterate     0  f=        -2070  |proj g|=       221.81

iterations 7
function evaluations 10
segments explored during Cauchy searches 8
BFGS updates skipped 0
active bounds at final generalized Cauchy point 0
norm of the final projected gradient 0.058588
final function value -2214.08

F = -2214.08
final  value -2214.079465 
converged
=== ncbc ===
Estimated Parameters:
[1] 0.4227626 0.2076383 0.4527587
Final Value (NLL): -2517.384 
Log-Likelihood: 0 
AIC: -5028.769 
BIC: -5009.757 
K-S Statistic: 0.06268806 
P-Value: 0.1214463 
Standard Errors:
[1] 0.008289319 0.016598653 0.004964320

=== norm ===
Estimated Parameters:
[1] 0.5240086 0.1200882
Final Value (NLL): -2925.666 
Log-Likelihood: 0 
AIC: -5847.332 
BIC: -5834.658 
K-S Statistic: 0.07429905 
P-Value: 0.1430778 
Standard Errors:
[1] 0.001856462 0.001313373

=== cauchy ===
Estimated Parameters:
[1] 0.550

agg_record_180398614 
                   2